In [1]:
#import the library
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import seaborn as sns
import logging
#import tensorflow as tf
import os
import anndata

import importlib
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

### Get the at least 1% expressed genes for NicheNet to determine what receptor is expressed

In [ ]:
def get_expressed_genes(celltypes, adata, pct=0.01):
    expressed_genes = {}

    for celltype in celltypes:
        adata_sub = adata[adata.obs['tissue+celltype'] == celltype]

        gene_expression = (adata_sub.layers['counts'] > 0).sum(axis=0) / adata_sub.n_obs
        if isinstance(gene_expression, np.matrix) or hasattr(gene_expression, "A1"):
            gene_expression = np.array(gene_expression).flatten()
        expressed_genes[celltype] = adata.var_names[gene_expression >= pct].tolist()
    
    return expressed_genes

In [ ]:
save_names = ['CD4DEG_IEL_vs_LP_1','CD4DEG_LP_vs_L_1','CD8DEG_IEL_vs_LP_1','CD8DEG_LP_vs_L_1',
              'CD4DEG_IEL_vs_LP_2','CD4DEG_LP_vs_L_2','CD8DEG_IEL_vs_LP_2','CD8DEG_LP_vs_L_2']

In [ ]:
celltype_names =  ['IEL TCRab CD4 TRM','LP TCRab CD4 TRM',
                                       'IEL TCRab CD8ab TRM',
                                       'LP TCRab CD8ab TRM',
                                       'LP TCRab CD4 TRM',
                                       'L TCRab CD4 TRM',
                                       'LP TCRab CD8ab TRM',
                                       'L TCRab CD8ab TRM']

In [ ]:
expressed_genes = get_expressed_genes(celltype_names, adata_all)

In [ ]:
for i, key in enumerate(save_names): 
    pd.DataFrame(expressed_genes[celltype_names[i]]).to_csv('E:/AAA_Labwork/T cells/v3/'+key+'_receptors.csv')

In [ ]:
expressed_genes = get_expressed_genes(['TRM'],adata_all)
pd.DataFrame(expressed_genes['TRM']).to_csv('E:/AAA_Labwork/T cells/v3/TRM_background_genes.csv')